In [ ]:
import cv2 as cv
# per gestire lo scatto della foto dal video
from cv2 import VideoCapture
# per caricare il modello CNN
from tensorflow import keras
# per pulire output console
from IPython.display import clear_output
# per operazioni con array
import numpy as np
import os # per gestire i path
import pickle
import tensorflow as tf

In [ ]:
def start_demo(face_detector, model, dict_names):
    """
    Visualizza una finestra con la riproduzione streaming catturata dalla camera principale
    del pc. Ad ogni frame viene quindi applicato il face detector e, se presente un volto,
    quest'ultimo viene passato al modello per riconoscere la persona.
    
    :param cv2.CascadeClassifier face_detector: Detector per il riconoscimento del volto
    :param keras.engine.functional.Functional model: Modello keras per la predizione
    :param str{} dict_names: Dizionario che associa nomi e numeri
    """
        
    # nel caso ci siano errori chiudo la finestra in automatico
    try:
        # initialize the camera
        cam = VideoCapture(0)

        while True:
            # acquisisco l'immagine dalla cam
            s, frame = cam.read()
            
            # se l'acquisizione non è andata a buon fine interrompo
            if not s:
                break
            else:
                # converto l'immagine in b/n
                img_bw = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
                
                # applico il face detector sull'immagine in b/n
                faces = face_detector.detectMultiScale(img_bw, 1.1, 8)

                for (x,y,w,h) in faces:
                    # visualizzo il rettangolo di detection
                    cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                    
                    # estraggo il crop
                    crop = frame[y:y+h, x:x+w]
                    # converto il crop a colori RGB
                    crop = cv.cvtColor(crop, cv.COLOR_BGR2RGB)
                    # riscalo il crop in 224x224
                    crop = cv.resize(crop, (224, 224)) 
                    # aggiungo una dimensione
                    crop = np.expand_dims(crop, axis = 0)
                     
                    # applico il modello al crop
                    y_pred = model.predict(crop)
                    
                    # estraggo la predizione migliore
                    y_pred_max = np.argmax(y_pred)
                    
                    # ottengo la probabilità della previsione
                    y_pred = y_pred[0][y_pred_max]
                    
                    # se il modello predice con probabilità inferiore allora è incerto e quindi non visualizzo il testo
                    if y_pred > 0.0:
                        # visualizzo la previsione
                        cv.putText(frame, 
                               ("Person: " + str(dict_names[y_pred_max])) + ' - ' + str(np.round(y_pred,3)), 
                               ((x, y-5)), 
                                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                    
            
            # visualizzo la finestra
            cv.imshow('FaceRecognition', frame)

            # Termino il ciclo se viene premuto Q
            if cv.waitKey(20) & 0xFF == ord('q'):
                break
        
        # rilascio la cam e chiudo la finestra se il ciclo è finito
        cam.release()
        cv.destroyAllWindows()
        
    except: # rilascio la cam e chiudo la finestra
        cam.release()
        cv.destroyAllWindows()

In [ ]:
# carico il modello di face detection pre-addestrato
face_detector = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
# carico il modello pre-trainato
loaded_model = keras.models.load_model('FaceRecBest.h5')

In [ ]:
# definisco un dizionario con l'associazione Label-Nome
dict_name = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
start_demo(face_detector, loaded_model, dict_name)

In [ ]:
# per uscire dalla face-rec premere Q